In [2]:
def getDataFiles(list_filename):
    return [line.rstrip() for line in open(list_filename)]

In [4]:
BASE_DIR = os.path.dirname(os.path.abspath('train.py'))

TRAIN_FILES = getDataFiles( \
os.path.join(BASE_DIR, 'data/modelnet40_ply_hdf5_2048/train_files.txt'))

train_file_idxs = np.arange(0, len(TRAIN_FILES))

current_data, current_label = provider.loadDataFile(TRAIN_FILES[train_file_idxs[0]])

/home/jackjill/Desktop/Machine Vision/pointnet-master/provider.py:91: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f = h5py.File(h5_filename)


In [5]:
print(type(current_data)) 
print(current_data.shape)
print(current_data[0])

#2048 images in one file. each image has (2048, 3) array with the coordinates.

<class 'numpy.ndarray'>
(2048, 2048, 3)
[[ 0.09804384 -0.16375712 -0.88815236]
 [-0.23270538  0.20258272  0.9004929 ]
 [-0.2567441  -0.25970268 -0.49264464]
 ...
 [-0.39915243  0.11317458 -0.87155205]
 [-0.39915243  0.06148027 -0.857612  ]
 [ 0.05074174  0.04413288  0.13229933]]


In [8]:
import math

import sys
if sys.version_info >= (3,0):
    from functools import reduce

In [9]:
def euler2mat(z=0, y=0, x=0):
    Ms = []
    if z:
        cosz = math.cos(z)
        sinz = math.sin(z)
        Ms.append(np.array(
                [[cosz, -sinz, 0],
                 [sinz, cosz, 0],
                 [0, 0, 1]]))
    if y:
        cosy = math.cos(y)
        siny = math.sin(y)
        Ms.append(np.array(
                [[cosy, 0, siny],
                 [0, 1, 0],
                 [-siny, 0, cosy]]))
    if x:
        cosx = math.cos(x)
        sinx = math.sin(x)
        Ms.append(np.array(
                [[1, 0, 0],
                 [0, cosx, -sinx],
                 [0, sinx, cosx]]))
    if Ms:
        return reduce(np.dot, Ms[::-1]) 
    return np.eye(3)
    
#Rotation matrix is returned
    #http://en.wikipedia.org/wiki/Rotation_matrix    

In [12]:
def draw_point_cloud(input_points, canvasSize=500, space=200, diameter=25,
                     xrot=0, yrot=0, zrot=0, switch_xyz=[0,1,2], normalize=True):
    """ Render point cloud to image with alpha channel.
        Input:
            points: Nx3 numpy array (+y is up direction)
        Output:
            gray image as numpy array of size canvasSizexcanvasSize
    """
    image = np.zeros((canvasSize, canvasSize))
    if input_points is None or input_points.shape[0] == 0:
        return image

    points = input_points[:, switch_xyz]
    M = euler2mat(zrot, yrot, xrot)
    #print(M) #******
    points = (np.dot(M, points.transpose())).transpose()
    #print(points.shape) #******
    # Normalize the point cloud
    # We normalize scale to fit points in a unit sphere
    if normalize:
        centroid = np.mean(points, axis=0)
        points -= centroid
        furthest_distance = np.max(np.sqrt(np.sum(abs(points)**2,axis=-1)))
        points /= furthest_distance

    # Pre-compute the Gaussian disk
    radius = (diameter-1)/2.0
    disk = np.zeros((diameter, diameter))
    for i in range(diameter):
        for j in range(diameter):
            if (i - radius) * (i-radius) + (j-radius) * (j-radius) <= radius * radius:
                disk[i, j] = np.exp((-(i-radius)**2 - (j-radius)**2)/(radius**2))
    mask = np.argwhere(disk > 0) #the indexes of the places with values > 0
    dx = mask[:, 0] #x coordinates with positive values
    dy = mask[:, 1] #y coordinates with positive values
    dv = disk[disk > 0] #positive values in the disk
    
    # Order points by z-buffer
    zorder = np.argsort(points[:, 2]) 
    points = points[zorder, :]
    points[:, 2] = (points[:, 2] - np.min(points[:, 2])) / (np.max(points[:, 2] - np.min(points[:, 2])))
    max_depth = np.max(points[:, 2])
       
    for i in range(points.shape[0]):
        j = points.shape[0] - i - 1 #from 2047 to 0
        x = points[j, 0] #from largest to smallest value
        y = points[j, 1]
        xc = canvasSize/2 + (x*space)
        yc = canvasSize/2 + (y*space)
        xc = int(np.round(xc))
        yc = int(np.round(yc))
        
        px = dx + xc
        py = dy + yc
        
        image[px, py] = image[px, py] * 0.7 + dv * (max_depth - points[j, 2]) * 0.3
    
    image = image / np.max(image)
    return image

In [13]:
def point_cloud_three_views(points):
    """ input points Nx3 numpy array (+y is up direction).
        return an numpy array gray image of size 500x1500. """ 
    # +y is up direction
    # xrot is azimuth
    # yrot is in-plane
    # zrot is elevation
    #print("in")
    img1 = draw_point_cloud(points, zrot=110/180.0*np.pi, xrot=45/180.0*np.pi, yrot=0/180.0*np.pi)
    img2 = draw_point_cloud(points, zrot=70/180.0*np.pi, xrot=135/180.0*np.pi, yrot=0/180.0*np.pi)
    img3 = draw_point_cloud(points, zrot=180.0/180.0*np.pi, xrot=90/180.0*np.pi, yrot=0/180.0*np.pi)
    image_large = np.concatenate([img1, img2, img3], 1) #concatenating three images into one
    return image_large

In [16]:
import sys
import scipy.misc
import imageio

In [14]:
output_img = point_cloud_three_views(np.squeeze(current_data[0, :, :]))

In [17]:
imageio.imwrite('sofa_train.jpg', output_img)

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


In [1]:
import h5py
import pandas as pd
import numpy as np
import os